In [5]:
import numpy as np
import pandas as pd
import os, sys
from scipy import signal

CURRENT_DIR = os.path.dirname(os.getcwd())
sys.path.append(os.path.dirname(CURRENT_DIR+'/func'))
from func.photonic_func import Photonic

from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
init_notebook_mode(connected=True)  # for Jupyter Lab notebook

nm = 1e-9
nsec = 1e-9
c = 2.99792458e8  # [m/s] Speed of light
hc = 1.987820871E-025  # [J * m / photon] Energy of photon with wavelength m

# Create WP with n shutters per cycle

In [20]:
photonic = Photonic(config='Cfg3')

freq = 60e6
cycle_time = 1 / freq
shutter_per_cycle = 3

light_rise = photonic.op.light_rise_nsec * nsec
light_fall = photonic.op.light_fall_nsec * nsec
light_width = photonic.op.light_width_nsec * nsec
shutter_rise = photonic.op.shutter_rise_nsec * nsec
shutter_fall = photonic.op.shutter_fall_nsec * nsec
shutter_width = photonic.op.shutter_width_nsec * nsec
shutter_delay = photonic.op.shutter_delay_nsec * nsec
y1, t1 = photonic.generate_pulse(rise=light_rise, fall=light_fall, width=light_width, smooth=True)
y2, t2 = photonic.generate_pulse(delay=shutter_delay, rise=shutter_rise, fall=shutter_fall, width=shutter_width, smooth=True)
y3, t3 = photonic.conv_light_shutter(t_light=t1, y_light=y1, t_shutter=t2, y_shutter=y2)

shutters = dict()
for i in range(shutter_per_cycle + 1):
    sh_delay = i * cycle_time / shutter_per_cycle
    shutters['sh'+str(i)] = np.stack((t3 + sh_delay, y3)).T


shutters['sh0'].shape  

(430, 2)

In [21]:
trace = [1,2,3,4]
for i in range(shutter_per_cycle):
    trace[i] = go.Scatter(x=shutters['sh'+str(i)][:,0], y=shutters['sh'+str(i)][:,1],
                          mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                          name='sh'+str(i))
    
data = trace

layout = dict(title='WP',
              xaxis=dict(title='time delay [sec]',type='linear'),  # Select 'log' or 'linear'
              yaxis=dict(title='Signal',type='linear'),  # Select 'log' or 'linear'
              template='plotly_dark')

iplot(dict(data=data, layout=layout))